In [1]:
!pip install git+https://github.com/qubvel/segmentation_models.pytorch

  Cloning https://github.com/qubvel/segmentation_models.pytorch to /tmp/pip-req-build-5g93chnf
  Running command git clone --filter=blob:none --quiet https://github.com/qubvel/segmentation_models.pytorch /tmp/pip-req-build-5g93chnf
  Resolved https://github.com/qubvel/segmentation_models.pytorch to commit e76ed01d49ff986be065fcc3c764255ce512cf16
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 83.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 65.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:

In [2]:
!pip install lightning

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 824.2/824.2 kB 391.4 kB/s eta 0:00:0000:01


In [3]:
!pip install rasterio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 94.5 MB/s eta 0:00:00:00:0100:01


In [4]:
import os
import torch
import numpy as np
import pandas as pd
import rasterio
import pytorch_lightning as pl
import segmentation_models_pytorch as smp


from sklearn.model_selection import KFold
from torch.utils.data import Dataset, DataLoader
from torch.optim import lr_scheduler

## Dataset

In [5]:
class CAS_Landslide_Dataset(Dataset):
    def __init__(self, images_dir, masks_dir, indices=None, augmentation=None):
        all_ids = sorted(os.listdir(images_dir))
        if indices is not None:
            self.ids = [all_ids[i] for i in indices]
        else:
            self.ids = all_ids
        self.images_fps = [os.path.join(images_dir, fname) for fname in self.ids]
        self.masks_fps = [os.path.join(masks_dir, fname) for fname in self.ids]
        self.augmentation = augmentation

    def __getitem__(self, i):
        with rasterio.open(self.images_fps[i]) as src_img:
            image = src_img.read()  # (C, H, W)
        with rasterio.open(self.masks_fps[i]) as src_mask:
            mask = src_mask.read(1)  # (H, W)

        image = np.transpose(image, (1, 2, 0))  # (H, W, C)
        mask = (mask > 0).astype("float32")
        mask = np.expand_dims(mask, axis=-1)

        if self.augmentation:
            augmented = self.augmentation(image=image, mask=mask)
            image, mask = augmented["image"], augmented["mask"]

        image = image.transpose(2, 0, 1)
        mask = mask.transpose(2, 0, 1)
        return image, mask

    def __len__(self):
        return len(self.ids)

In [6]:
def get_loss_fn(loss_name):
    if loss_name == "DiceLoss":
        return smp.losses.DiceLoss(smp.losses.BINARY_MODE, log_loss=False, from_logits=True)
    elif loss_name == "JaccardLoss":
        return smp.losses.JaccardLoss(smp.losses.BINARY_MODE, log_loss=False, from_logits=True)
    elif loss_name == "TverskyLoss":
        return smp.losses.TverskyLoss(smp.losses.BINARY_MODE, log_loss=False, from_logits=True, alpha=0.5, beta=0.5, gamma=1.0)
    elif loss_name == "FocalLoss":
        return smp.losses.FocalLoss(smp.losses.BINARY_MODE, gamma=2.0)
    elif loss_name == "SoftBCE":
        return smp.losses.SoftBCEWithLogitsLoss()
    elif loss_name == "MCCLoss":
        return smp.losses.MCCLoss()
    else:
        raise ValueError(f"Unknown loss function: {loss_name}")

## Model

In [7]:
import lightning as L

class CASLandslideMappingModel(L.LightningModule):
    def __init__(
        self, arch, encoder_name, encoder_weights, in_channels, out_classes, learning_rate, loss_function_name, **kwargs
    ):
        super().__init__()
        self.model = smp.create_model(
            arch,
            encoder_name=encoder_name,
            encoder_weights=encoder_weights,
            in_channels=in_channels,
            classes=out_classes,
            **kwargs,
        )
        self.learning_rate = learning_rate
        # preprocessing parameteres for image
        params = smp.encoders.get_preprocessing_params(encoder_name)
        self.register_buffer("std", torch.tensor(params["std"]).view(1, 3, 1, 1))
        self.register_buffer("mean", torch.tensor(params["mean"]).view(1, 3, 1, 1))

        # get loss function
        self.loss_fn = get_loss_fn(loss_function_name)

        self.training_step_outputs = []
        self.validation_step_outputs = []
        self.test_step_outputs = []

        # Metric logging for CSV
        self.metric_log_path = "assets/training_metrics.csv"
        self.logged_metrics = {
            "epoch": [],
            "stage": [],
            "loss": [],
            "per_image_iou": [],
            "dataset_iou": [],
            "f1_score": [],
            "f2_score": [],
            "accuracy": [],
            "recall": [],
            "precision": [],
        }

    def forward(self, image):
        # normalize image here
        image = (image - self.mean) / self.std
        mask = self.model(image)
        return mask

    def shared_step(self, batch, stage):
        image, mask = batch

        # Shape of the image should be (batch_size, num_channels, height, width)
        # if you work with grayscale images, expand channels dim to have [batch_size, 1, height, width]
        assert image.ndim == 4

        # Check that image dimensions are divisible by 32,
        # encoder and decoder connected by `skip connections` and usually encoder have 5 stages of
        # downsampling by factor 2 (2 ^ 5 = 32); e.g. if we have image with shape 65x65 we will have
        # following shapes of features in encoder and decoder: 84, 42, 21, 10, 5 -> 5, 10, 20, 40, 80
        # and we will get an error trying to concat these features
        h, w = image.shape[2:]
        assert h % 32 == 0 and w % 32 == 0

        assert mask.ndim == 4

        # Check that mask values in between 0 and 1, NOT 0 and 255 for binary segmentation
        assert mask.max() <= 1.0 and mask.min() >= 0

        logits_mask = self.forward(image)

        # Predicted mask contains logits, and loss_fn param `from_logits` is set to True
        loss = self.loss_fn(logits_mask, mask)

        # Lets compute metrics for some threshold
        # first convert mask values to probabilities, then
        # apply thresholding
        prob_mask = logits_mask.sigmoid()
        pred_mask = (prob_mask > 0.5).float()

        # We will compute IoU metric by two ways
        #   1. dataset-wise
        #   2. image-wise
        # but for now we just compute true positive, false positive, false negative and
        # true negative 'pixels' for each image and class
        # these values will be aggregated in the end of an epoch
        tp, fp, fn, tn = smp.metrics.get_stats(
            pred_mask.long(), mask.long(), mode="binary"
        )
        return {
            "loss": loss,
            "tp": tp,
            "fp": fp,
            "fn": fn,
            "tn": tn,
        }

    def shared_epoch_end(self, outputs, stage):
        tp = torch.cat([x["tp"] for x in outputs])
        fp = torch.cat([x["fp"] for x in outputs])
        fn = torch.cat([x["fn"] for x in outputs])
        tn = torch.cat([x["tn"] for x in outputs])

        per_image_iou = smp.metrics.iou_score(
            tp, fp, fn, tn, reduction="micro-imagewise"
        )
        dataset_iou = smp.metrics.iou_score(tp, fp, fn, tn, reduction="micro")
        f1_score = smp.metrics.f1_score(tp, fp, fn, tn, reduction="micro")
        f2_score = smp.metrics.fbeta_score(tp, fp, fn, tn, beta=2, reduction="micro")
        accuracy = smp.metrics.accuracy(tp, fp, fn, tn, reduction="macro")
        recall = smp.metrics.recall(tp, fp, fn, tn, reduction="micro-imagewise")
        precision = smp.metrics.precision(tp, fp, fn, tn, reduction="micro")

        metrics = {
            f"{stage}_per_image_iou": per_image_iou,
            f"{stage}_dataset_iou": dataset_iou,
            f"{stage}_f1_score": f1_score,
            f"{stage}_f2_score": f2_score,
            f"{stage}_accuracy": accuracy,
            f"{stage}_recall": recall,
            f"{stage}_precision": precision,
        }

        self.log_dict(metrics, prog_bar=True)
        avg_loss = torch.stack([x["loss"] for x in outputs]).mean()

        # Store current stage metrics
        self.logged_metrics["epoch"].append(self.current_epoch)
        self.logged_metrics["stage"].append(stage)
        self.logged_metrics["per_image_iou"].append(per_image_iou.item())
        self.logged_metrics["dataset_iou"].append(dataset_iou.item())
        self.logged_metrics["f1_score"].append(f1_score.item())
        self.logged_metrics["f2_score"].append(f2_score.item())
        self.logged_metrics["accuracy"].append(accuracy.item())
        self.logged_metrics["recall"].append(recall.item())
        self.logged_metrics["precision"].append(precision.item())
        self.logged_metrics["loss"].append(avg_loss.item())

        # Save just this stage's metrics to CSV
        self.save_epoch_metrics_to_csv()

    def save_epoch_metrics_to_csv(self):
        if not self.logged_metrics["epoch"]:
            return

        df = pd.DataFrame(self.logged_metrics)
        file_exists = os.path.exists(self.metric_log_path)

        if not file_exists:
            df.to_csv(self.metric_log_path, index=False)
        else:
            df.to_csv(self.metric_log_path, mode="a", header=False, index=False)

        # Clear after saving this stage
        for key in self.logged_metrics:
            self.logged_metrics[key].clear()

    def training_step(self, batch, batch_idx):
        train_loss_info = self.shared_step(batch, "train")
        # append the metics of each step to the
        self.training_step_outputs.append(train_loss_info)
        return train_loss_info

    def on_train_epoch_end(self):
        self.shared_epoch_end(self.training_step_outputs, "train")
        # empty set output list
        self.training_step_outputs.clear()
        return

    def validation_step(self, batch, batch_idx):
        valid_loss_info = self.shared_step(batch, "valid")
        self.validation_step_outputs.append(valid_loss_info)
        return valid_loss_info

    def on_validation_epoch_end(self):
        self.shared_epoch_end(self.validation_step_outputs, "valid")
        self.validation_step_outputs.clear()
        return

    def test_step(self, batch, batch_idx):
        test_loss_info = self.shared_step(batch, "test")
        self.test_step_outputs.append(test_loss_info)
        return test_loss_info

    def on_test_epoch_end(self):
        self.shared_epoch_end(self.test_step_outputs, "test")
        # empty set output list
        self.test_step_outputs.clear()
        return

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
            optimizer, T_max=10 * 100, eta_min=1e-5
        )
        return {
            "optimizer": optimizer,
            "lr_scheduler": {
                "scheduler": scheduler,
                "interval": "step",
                "frequency": 1,
            },
        }

## Config

In [8]:
# ------------
# Config
# ------------
NUM_FOLDS = 5
BATCH_SIZE = 8
NUM_WORKERS = 4
MAX_EPOCHS = 1
METRIC_CSV_PATH = "./fold_metrics.csv"
# model args
MODEL_CONFIG = {
    "arch": "Unet",
    "encoder_name": "densenet161",
    "encoder_weights": "imagenet",
    "in_channels": 3,
    "out_classes": 1,
    "learning_rate": 1e-3,
    "loss_function_name": "FocalLoss",  # custom from get_loss_fn
}

# data dirs
IMAGES_DIR = "/kaggle/input/cas-landslide-satellite-cross-validation/CAS_Landslide_Dataset_Satellite_Cross_Validation/img/"
MASKS_DIR = "/kaggle/input/cas-landslide-satellite-cross-validation/CAS_Landslide_Dataset_Satellite_Cross_Validation/mask/"

In [9]:
# ---------------- Helper Functions ----------------
def validate(trainer, model, valid_loader, fold):
    metrics = trainer.validate(model, dataloaders=valid_loader, verbose=True)
    return metrics[0]  # Lightning returns a list of dicts

def test(trainer, model, test_loader, fold):
    metrics = trainer.test(model, dataloaders=test_loader, verbose=True)
    return metrics[0]

In [10]:
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
early_stopping = EarlyStopping(
    monitor="valid_dataset_iou",
    mode="max",
    patience=30,
)

In [13]:
def run_cross_validation():
    # Load all .tif filenames beforehand
    all_ids = sorted(os.listdir(IMAGES_DIR))
    kfold = KFold(n_splits=NUM_FOLDS, shuffle=True, random_state=42)
    summary_rows = []
    for fold, (train_idx, val_idx) in enumerate(kfold.split(all_ids)):
        print(f"\n🔁 Fold {fold + 1}/{NUM_FOLDS}")

        # Pass indices directly into your dataset class
        train_dataset = CAS_Landslide_Dataset(IMAGES_DIR, MASKS_DIR, indices=train_idx)
        test_dataset = CAS_Landslide_Dataset(IMAGES_DIR, MASKS_DIR, indices=val_idx)

        train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
        test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

        model = CASLandslideMappingModel(**MODEL_CONFIG)

        trainer = L.Trainer(
            accelerator="gpu" if torch.cuda.is_available() else "cpu",
            max_epochs=MAX_EPOCHS,
            callbacks=[early_stopping],
            log_every_n_steps=10,
        )

        trainer.fit(model, train_loader, test_loader)

        print(f"✅ Fold {fold + 1}")
        test_metrics = test(trainer, model, test_loader, fold + 1)

        row = {
            "fold": fold + 1,
            **test_metrics # flatten metrics into summary row
        }
        summary_rows.append(row)

    # Save CSV summary of all folds
    if summary_rows:
        df = pd.DataFrame(summary_rows)
        df.to_csv("./summary_cross_validation.csv", index=False)
        print(f"\n📊 Fold summary saved to: {SUMMARY_CSV_PATH}")

In [ ]:
os.makedirs("assets", exist_ok=True)
run_cross_validation()


🔁 Fold 1/5


INFO: 💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO: GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name    | Type      | Params | Mode 
----------------------------------------------
0 | model   | Unet      | 38.7 M | train
1 | loss_fn | FocalLoss | 0      | train
----------------------------------------------
38.7 M    Trainable params
0         Non-trainable params
38.7 M    Total params
154.953   Total estimated model params size (MB)
647       Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/rasterio/__init__.py:356: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
/usr/local/lib/python3.11/dist-packages/rasterio/__init__.py:356: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
/usr/local/lib/python3.11/dist-packages/rasterio/__init__.py:356: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
/usr/local/lib/python3.11/dist-packages/rasterio/__init__.py:356: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


Training: |          | 0/? [00:00<?, ?it/s]

...

INFO: 💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO: GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name    | Type      | Params | Mode 
----------------------------------------------
0 | model   | Unet      | 38.7 M | train
1 | loss_fn | FocalLoss | 0      | train
----------------------------------------------
38.7 M    Trainable params
0         Non-trainable params
38.7 M    Total params
154.953   Total estimated model params size (MB)
647       Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/rasterio/__init__.py:356: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
/usr/local/lib/python3.11/dist-packages/rasterio/__init__.py:356: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
/usr/local/lib/python3.11/dist-packages/rasterio/__init__.py:356: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
/usr/local/lib/python3.11/dist-packages/rasterio/__init__.py:356: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


Training: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/rasterio/__init__.py:356: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
/usr/local/lib/python3.11/dist-packages/rasterio/__init__.py:356: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
/usr/local/lib/python3.11/dist-packages/rasterio/__init__.py:356: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
/usr/local/lib/python3.11/dist-packages/rasterio/__init__.py:356: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
